In [1]:
from octis.models.NeuralLDA import NeuralLDA
from octis.models.LDA import LDA
from octis.models.NMF_scikit import NMF_scikit
from octis.models.ETM import ETM
from octis.dataset.dataset import Dataset
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.models.CTM import CTM
from octis.dataset.dataset import Dataset
from octis.optimization.optimizer import Optimizer
from skopt.space.space import Real, Categorical, Integer
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.dataset.dataset import Dataset
from skopt.space.space import Real, Categorical, Integer
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.optimization.optimizer import Optimizer
optimizer=Optimizer()
import time
import en_core_web_sm
nlp = en_core_web_sm.load()
dataset = Dataset()
dataset.load_custom_dataset_from_folder("pre_Flower_stop")

/sw/arch/RHEL8/EB_production/2022/software/Python/3.10.4-GCCcore-11.3.0/lib/python3.10/site-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [10]:
#Creating a list of models with optimal parameters, found with OCTIS
model_neural_lda = NeuralLDA(num_topics=10, num_neurons=150, num_layers=2, dropout=0.46174495687047284)
model_NMF = NMF_scikit(num_topics=10, alpha=0.0002871719181692572)
model_LDA = LDA(num_topics=10, alpha=0.0002871719181692572)
model_CTM = CTM(num_topics=10, bert_path="all-mpnet-base-v2", activation='sigmoid', dropout=0.6514267016354713, num_layers=3, num_neurons=200)
model_ETM = ETM(num_topics=10, activation='softplus', dropout=0.4927880253307994, t_hidden_size=276)
model_list = [model_neural_lda]#, model_CTM, model_ETM, model_NMF, model_LDA]

In [11]:
model_dict = {}
for model in model_list:
    # Train the model using default partitioning choice
    output = model.train_model(dataset)
    model_dict[model]=output
    #print(*list(output.keys()), sep="\n") # Print the output identifiers

Epoch: [1/100]	Samples: [32364/3236400]	Train Loss: 511.87890365500635	Time: 0:00:06.754961
Epoch: [1/100]	Samples: [6936/693600]	Validation Loss: 475.6187334423659	Time: 0:00:00.967693
Epoch: [2/100]	Samples: [64728/3236400]	Train Loss: 498.16704939805805	Time: 0:00:06.121739
Epoch: [2/100]	Samples: [6936/693600]	Validation Loss: 472.07170610086865	Time: 0:00:00.974835
Epoch: [3/100]	Samples: [97092/3236400]	Train Loss: 497.87904565547365	Time: 0:00:06.125191
Epoch: [3/100]	Samples: [6936/693600]	Validation Loss: 472.5453841089515	Time: 0:00:00.969036
Epoch: [4/100]	Samples: [129456/3236400]	Train Loss: 496.96042073416834	Time: 0:00:05.860138
Epoch: [4/100]	Samples: [6936/693600]	Validation Loss: 470.3990854441501	Time: 0:00:00.974212
Epoch: [5/100]	Samples: [161820/3236400]	Train Loss: 496.1814959088416	Time: 0:00:06.062140
Epoch: [5/100]	Samples: [6936/693600]	Validation Loss: 470.71674280925606	Time: 0:00:00.967911
Epoch: [6/100]	Samples: [194184/3236400]	Train Loss: 495.9055454290

In [12]:
for model in model_dict.keys():
    for t in model_dict[model]['topics']:
        print(" ".join(t))

flowers ale colourless unusual ner february its ditch showy ulcer
des bitterish druggist spangle hinder news ignorance confound par belt
call they people thick shrub taste away yellow green aromatic
fragrant flower perfume leave smell man like white long scent
indifferent handful genus semi robert inhabit upon sicken vividly perch
fetid etc olfactory citron record valuable bathe petiole public introduce
glabrous reek eafily pubescent vanilla diftance tubular swamp divinity enlarge
non background moan permanent resist decompose sixteen purgative succulent model
herbaceous expanse chaste grieve pensive chemical coriaceous cheap relic gallant
climbing pedicel deliciously cart expense spotless specific brilliantly java puff


In [13]:
model_dict

{<octis.models.NeuralLDA.NeuralLDA at 0x14bfaed39ab0>: {'topics': [['flowers',
    'ale',
    'colourless',
    'unusual',
    'ner',
    'february',
    'its',
    'ditch',
    'showy',
    'ulcer'],
   ['des',
    'bitterish',
    'druggist',
    'spangle',
    'hinder',
    'news',
    'ignorance',
    'confound',
    'par',
    'belt'],
   ['call',
    'they',
    'people',
    'thick',
    'shrub',
    'taste',
    'away',
    'yellow',
    'green',
    'aromatic'],
   ['fragrant',
    'flower',
    'perfume',
    'leave',
    'smell',
    'man',
    'like',
    'white',
    'long',
    'scent'],
   ['indifferent',
    'handful',
    'genus',
    'semi',
    'robert',
    'inhabit',
    'upon',
    'sicken',
    'vividly',
    'perch'],
   ['fetid',
    'etc',
    'olfactory',
    'citron',
    'record',
    'valuable',
    'bathe',
    'petiole',
    'public',
    'introduce'],
   ['glabrous',
    'reek',
    'eafily',
    'pubescent',
    'vanilla',
    'diftance',
    'tubular'

In [14]:
# Initialize metric
coherence = Coherence(texts=dataset.get_corpus(), measure = 'c_npmi')
diversity = TopicDiversity(topk=10)

In [15]:
model_dict.keys()

dict_keys([<octis.models.NeuralLDA.NeuralLDA object at 0x14bfaed39ab0>])

In [16]:
# Retrieve metrics score
for model in model_dict.keys():
    output = model_dict[model]
    diversity_score = diversity.score(output)
    print("Used model: "+str(model))
    print("Topic diversity: "+str(diversity_score))

    coherence_score = coherence.score(output)
    print("Coherence: "+str(coherence_score))

Used model: <octis.models.NeuralLDA.NeuralLDA object at 0x14bfaed39ab0>
Topic diversity: 1.0
Coherence: -0.3024270202896217
